In [1]:
import numpy as np

layout = np.array([])
with open("input_day18.txt","r") as f:
    fileLines = f.readlines()
    xSize = fileLines[0].find("\n")
    flatLayout=np.array([])
    for line in fileLines:
        flatLayout = np.append(flatLayout,[s for s in line[:-1]])
    layout = np.reshape(flatLayout,(len(flatLayout)//xSize,xSize))
    
print(layout,xSize)

def getInitialPositionAndRemoveFromLayout(layout):
    for j in range(0,len(layout)):
        for i in range(0,len(layout[0])):
            if layout[j,i] == '@':
                layout[j,i] = '.'
                return (j,i)

def getAllKeys(layout):
    keys=[]
    for j in range(0,len(layout)):
        for i in range(0,len(layout[0])):
            if layout[j,i].isalpha() and layout[j,i].islower():
                keys.append(layout[j,i])
    return keys
            
def getOffsetFromDirection(direction):
    if direction == 1:
        return np.array([-1,0])
    elif direction == 2:
        return np.array([0,1])
    elif direction == 3:
        return np.array([1,0])
    elif direction == 4:
        return np.array([0,-1])
    else:
        return 0

# valid = not wall and not yet visited
def getValidNewPositions(layout, distanceMap, position):
    validNewPos=[]
    for direction in range(1,5):
        newPos = np.array(list(position))+getOffsetFromDirection(direction)
        if layout[tuple(newPos)] != '#' and distanceMap[tuple(newPos)] == -1:
            validNewPos.append(newPos)
    return validNewPos
            

def getReachableKeys(layout, position, collectedKeys):
    # get list of all reachable keys, together with their position and current distance
    distance = 0
    distanceMap=np.full_like(layout, -1,dtype=int)
    activePositions=[position]
    newPositions=[]
    reachableKeys = []
    while True:
        for pos in activePositions:
            distanceMap[tuple(pos)] = distance
            entry = layout[tuple(pos)]
            if entry.isalpha():
                if entry.islower():
                    #found key
                    if entry not in collectedKeys:
                        reachableKeys.append((entry, pos, distance))
                        continue
                if entry.isupper():
                    # found closed door
                    if entry.lower() not in collectedKeys:
                        # door can not be opened
                        continue
            
            newValidPositions = getValidNewPositions(layout, distanceMap, pos)
            for newPos in newValidPositions:
                newPositions.append(newPos)
 
        activePositions,newPositions=newPositions,[] 
        distance += 1
        if len(activePositions) == 0:
            return reachableKeys

def allKeysFound(availableKeys, collectedKeys):
    for key in availableKeys:
        if key not in collectedKeys:
            return False
    return True

def isIdentical(keys1, keys2):
    if keys1[-1] != keys2[-1]:
        return False
    for key in keys1:
        if key not in keys2:
            return False
    return True
        

def getShortestPathToAllKeys(layout, initialPosition, availableKeys):
    
    currentOptions=[(initialPosition, [], 0)]
    newOptions=[]
    foundSuccessfulOptions=[]
    iteration=1
    while True:
        print("Iteration ", iteration, " with number of options ", len(currentOptions))
        for option in currentOptions:
            #print(option)
            pos = option[0]
            collectedKeys = option[1]
            distance = option[2]
            
            nextKeys = getReachableKeys(layout, pos, collectedKeys)
            
            for nextKey in nextKeys:
                key = nextKey[0]
                keyPos = nextKey[1]
                newDistance = nextKey[2] + distance
                newCollectedKeys = collectedKeys + [key]
                if allKeysFound(availableKeys, newCollectedKeys):
                    foundSuccessfulOptions.append((newCollectedKeys, newDistance))
                    break
                else:
                    # check if we already have a better option or if this option is better than already found ones
                    #important to not overcrowed the options with already better ones
                    foundInList = False
                    for idx, alreadyFoundOption in enumerate(newOptions):
                        if isIdentical(newCollectedKeys, alreadyFoundOption[1]):
                            foundInList = True
                            if newDistance < alreadyFoundOption[2]:
                                newOptions[idx] = (keyPos, newCollectedKeys, newDistance)
                            break
                    if not foundInList:     
                        newOptions.append((keyPos, newCollectedKeys, newDistance))
                
        currentOptions, newOptions = newOptions, []
        iteration += 1
        
        if len(currentOptions) == 0:
            break
    
    print(foundSuccessfulOptions)
    
    shortestPath=100000000
    bestOption=foundSuccessfulOptions[0]
    for option in foundSuccessfulOptions:
        if option[1] < shortestPath:
            shortestPath = option[1]
            bestOption=option
    return bestOption
        
initialPosition = getInitialPositionAndRemoveFromLayout(layout)
availableKeys=getAllKeys(layout)
print(availableKeys, len(availableKeys))

shortestPath = getShortestPathToAllKeys(layout, initialPosition, availableKeys)
print("Solution: ", shortestPath[0], shortestPath[1])

[['#' '#' '#' ... '#' '#' '#']
 ['#' '.' '#' ... '.' '.' '#']
 ['#' 'X' '#' ... '#' '.' '#']
 ...
 ['#' '.' '#' ... '#' '.' '#']
 ['#' '.' '.' ... '.' '.' '#']
 ['#' '#' '#' ... '#' '#' '#']] 81
['r', 'c', 'j', 'g', 'n', 'i', 'e', 'y', 'z', 'o', 'b', 'u', 'v', 'd', 'a', 'f', 'm', 's', 't', 'k', 'h', 'w', 'p', 'q', 'x', 'l'] 26
Iteration  1  with number of options  1
Iteration  2  with number of options  4
Iteration  3  with number of options  14
Iteration  4  with number of options  28
Iteration  5  with number of options  52
Iteration  6  with number of options  90
Iteration  7  with number of options  156
Iteration  8  with number of options  268
Iteration  9  with number of options  422
Iteration  10  with number of options  571
Iteration  11  with number of options  641
Iteration  12  with number of options  600
Iteration  13  with number of options  506
Iteration  14  with number of options  436
Iteration  15  with number of options  409
Iteration  16  with number of options  392


In [21]:
# task 2

import numpy as np

layout = np.array([])
with open("input_day18.txt","r") as f:
    fileLines = f.readlines()
    xSize = fileLines[0].find("\n")
    flatLayout=np.array([])
    for line in fileLines:
        flatLayout = np.append(flatLayout,[s for s in line[:-1]])
    layout = np.reshape(flatLayout,(len(flatLayout)//xSize,xSize))
    

def setFourRobots(layout):
    for j in range(0,len(layout)):
        for i in range(0,len(layout[0])):
            if layout[j,i] == '@':
                layout[j-1,i-1] = '@'
                layout[j-1,i] = '#'
                layout[j-1,i+1] = '@'
                layout[j,i-1] = '#'
                layout[j,i] = '#'
                layout[j,i+1] = '#'
                layout[j+1,i-1] = '@'
                layout[j+1,i] = '#'
                layout[j+1,i+1] = '@'
                return

def getInitialPositionsAndRemoveFromLayout(layout):
    initialPositions = []
    for j in range(0,len(layout)):
        for i in range(0,len(layout[0])):
            if layout[j,i] == '@':
                layout[j,i] = '.'
                initialPositions.append(np.array([j,i]))
    return initialPositions

def getAllKeys(layout):
    keys=[]
    for j in range(0,len(layout)):
        for i in range(0,len(layout[0])):
            if layout[j,i].isalpha() and layout[j,i].islower():
                keys.append(layout[j,i])
    return keys
            
def getOffsetFromDirection(direction):
    if direction == 1:
        return np.array([-1,0])
    elif direction == 2:
        return np.array([0,1])
    elif direction == 3:
        return np.array([1,0])
    elif direction == 4:
        return np.array([0,-1])
    else:
        return 0

# valid = not wall and not yet visited
def getValidNewPositions(layout, distanceMap, position):
    validNewPos=[]
    for direction in range(1,5):
        newPos = np.array(list(position))+getOffsetFromDirection(direction)
        if layout[tuple(newPos)] != '#' and distanceMap[tuple(newPos)] == -1:
            validNewPos.append(newPos)
    return validNewPos
            

def getReachableKeys(layout, position, collectedKeys):
    # get list of all reachable keys, together with their position and current distance
    distance = 0
    distanceMap=np.full_like(layout, -1,dtype=int)
    activePositions=[position]
    newPositions=[]
    reachableKeys = []
    while True:
        for pos in activePositions:
            distanceMap[tuple(pos)] = distance
            entry = layout[tuple(pos)]
            if entry.isalpha():
                if entry.islower():
                    #found key
                    if entry not in collectedKeys:
                        reachableKeys.append((entry, pos, distance))
                        continue
                if entry.isupper():
                    # found closed door
                    if entry.lower() not in collectedKeys:
                        # door can not be opened
                        continue
            
            newValidPositions = getValidNewPositions(layout, distanceMap, pos)
            for newPos in newValidPositions:
                newPositions.append(newPos)
 
        activePositions,newPositions=newPositions,[] 
        distance += 1
        if len(activePositions) == 0:
            return reachableKeys

def allKeysFound(availableKeys, collectedKeys):
    for key in availableKeys:
        if key not in collectedKeys:
            return False
    return True

def hasIdenticalKeys(keys1, keys2):
    for key in keys1:
        if key not in keys2:
            return False
    return True
    
def hasIdenticalRobotPositions(pos1, pos2):
    for i in range(0,len(pos1)):
        if tuple(pos1[i]) != tuple(pos2[i]):
            return False
    return True

def getShortestPathToAllKeys(layout, initialPositions, availableKeys):
    
    currentOptions=[(initialPositions, [], 0)]
    newOptions=[]
    foundSuccessfulOptions=[]
    iteration=1
    while True:
        print("Iteration ", iteration, " with number of options ", len(currentOptions))
        for option in currentOptions:
            #print("Evaluating option ", option)
            collectedKeys = option[1]
            distance = option[2]
            for idx,robotPos in enumerate(option[0]):
            
                nextKeys = getReachableKeys(layout, robotPos, collectedKeys)
            
                for nextKey in nextKeys:
                    key = nextKey[0]
                    keyPos = nextKey[1]
                    newDistance = nextKey[2] + distance
                    newCollectedKeys = collectedKeys + [key]
                    if allKeysFound(availableKeys, newCollectedKeys):
                        foundSuccessfulOptions.append((newCollectedKeys, newDistance))
                        break
                    else:
                        # check if we already have a better option or if this option is better than already found ones
                        #important to not overcrowed the options with already better ones
                        foundInList = False
                        newRobotPositions = option[0].copy()
                        newRobotPositions[idx] = keyPos
                        for i,alreadyFoundOption in enumerate(newOptions):
                            if hasIdenticalKeys(newCollectedKeys, alreadyFoundOption[1]) \
                               and hasIdenticalRobotPositions(newRobotPositions, alreadyFoundOption[0]):
                                foundInList = True
                                if newDistance < alreadyFoundOption[2]:
                                    newOptions[i] = (newRobotPositions, newCollectedKeys, newDistance)
                                break
                        if not foundInList:     
                            newOptions.append((newRobotPositions, newCollectedKeys, newDistance))
                
        currentOptions, newOptions = newOptions, []
        iteration += 1
        
        if len(currentOptions) == 0:
            break
    
    print(foundSuccessfulOptions)
    
    shortestPath=foundSuccessfulOptions[1]
    bestOption=foundSuccessfulOptions[0]
    for option in foundSuccessfulOptions:
        if option[1] < shortestPath:
            shortestPath = option[1]
            bestOption=option
    return bestOption
        
    
setFourRobots(layout)
#print(layout)
initialPositions = getInitialPositionsAndRemoveFromLayout(layout)
print(initialPositions)
availableKeys=getAllKeys(layout)
print(availableKeys, len(availableKeys))

shortestPath = getShortestPathToAllKeys(layout, initialPositions, availableKeys)
print("Solution: ", shortestPath[0], shortestPath[1])

[array([39, 39]), array([39, 41]), array([41, 39]), array([41, 41])]
['r', 'c', 'j', 'g', 'n', 'i', 'e', 'y', 'z', 'o', 'b', 'u', 'v', 'd', 'a', 'f', 'm', 's', 't', 'k', 'h', 'w', 'p', 'q', 'x', 'l'] 26
Iteration  1  with number of options  1
Iteration  2  with number of options  4
Iteration  3  with number of options  8
Iteration  4  with number of options  14
Iteration  5  with number of options  25
Iteration  6  with number of options  42
Iteration  7  with number of options  76
Iteration  8  with number of options  141
Iteration  9  with number of options  248
Iteration  10  with number of options  383
Iteration  11  with number of options  505
Iteration  12  with number of options  575
Iteration  13  with number of options  591
Iteration  14  with number of options  593
Iteration  15  with number of options  619
Iteration  16  with number of options  647
Iteration  17  with number of options  643
Iteration  18  with number of options  614
Iteration  19  with number of options  599